In [15]:
import numpy as np
#from utils2 import *
from tensorflow.keras.layers import *
from tensorflow.keras import *
import tensorflow as tf

import matplotlib.pyplot as plot

# https://numpy.org/doc/stable/user/numpy-for-matlab-users.html
import numpy.matlib as M

from keras import backend as K


# Load MATLAB variables saved to the file data.mat. (Note: When saving arrays to data.mat in MATLAB/Octave,
# use a recent binary format. scipy.io.loadmat will create a dictionary with the saved arrays and further information.)
import scipy.io as io


#Input=io.loadmat(r"C:\Users\Jamal\OneDrive\Jamal\MATLAB Code\Our Papers\Matlabcodes__Dynamic Structure\DataSet\TrainingData.mat")
Input=io.loadmat(r"D:\Data Set\Data1\TrainingData.mat")
H_input=Input['TrainingData']
print(H_input.shape)

#Input=io.loadmat(r"C:\Users\Jamal\OneDrive\Jamal\MATLAB Code\Our Papers\Matlabcodes__Dynamic Structure\DataSet\TrainingSwitchMatrix.mat")
Input=io.loadmat(r"D:\Data Set\Data1\TrainingSwitchMatrix.mat")
Label=Input['TrainingSwitchMatrix']
print(Label.shape)
#Nt=Label.shape[1]
#K=Label.shape[2]
num_classes=Label.shape[1]

(66666, 256, 10, 3)
(66666, 2560)


In [16]:
# imperfect CSI is used to output the vrf
imperfect_CSI = tf.keras.Input(name='imperfect_CSI', shape=(H_input.shape[1:4]), dtype=tf.float32)
temp = BatchNormalization()(imperfect_CSI)
temp = Flatten()(temp)
#temp = BatchNormalization()(temp)
temp = Dense(num_classes, activation='relu')(temp)
#temp = BatchNormalization()(temp)
#temp = Dense(num_classes, activation='relu')(temp)
#temp = BatchNormalization()(temp)
output= Dense(num_classes, name='FC_layer4')(temp)
output=Activation('sigmoid')(output)
#output_reshape  = Reshape(( Nt, K))(output)
model = Model(inputs=[imperfect_CSI], outputs=output)

def cat_loss(y_true, y_pred):
    loss = K.mean(K.binary_crossentropy(y_true[:, 0:num_classes - 1], y_pred[:, 0:num_classes - 1]), axis=-1) \
             + K.mean(K.binary_crossentropy(y_true[:, num_classes:2 * num_classes - 1], y_pred[:, num_classes:2 * num_classes - 1]), axis=-1)
    return loss

adam = tf.keras.optimizers.Adam(learning_rate=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    
model.compile(optimizer='adam', 
              loss=tf.keras.losses.BinaryCrossentropy())
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 imperfect_CSI (InputLayer)  [(None, 256, 10, 3)]      0         
                                                                 
 batch_normalization_3 (Batc  (None, 256, 10, 3)       12        
 hNormalization)                                                 
                                                                 
 flatten_3 (Flatten)         (None, 7680)              0         
                                                                 
 dense_3 (Dense)             (None, 2560)              19663360  
                                                                 
 FC_layer4 (Dense)           (None, 2560)              6556160   
                                                                 
 activation_3 (Activation)   (None, 2560)              0         
                                                           

In [17]:
Label = tf.cast(Label, tf.float32)
model.fit(H_input, Label,batch_size=500, epochs=20)

Epoch 1/20
134/134 [==============================] - 204s 2s/step - loss: 0.1202
Epoch 2/20
134/134 [==============================] - 166s 1s/step - loss: 9.2365e-04
Epoch 3/20
134/134 [==============================] - 139s 1s/step - loss: 3.7706e-04
Epoch 4/20
134/134 [==============================] - 130s 969ms/step - loss: 2.0982e-04
Epoch 5/20
134/134 [==============================] - 136s 1s/step - loss: 1.3483e-04
Epoch 6/20
134/134 [==============================] - 120s 893ms/step - loss: 9.4336e-05
Epoch 7/20
134/134 [==============================] - 108s 807ms/step - loss: 6.9800e-05
Epoch 8/20
134/134 [==============================] - 104s 776ms/step - loss: 5.3750e-05
Epoch 9/20
134/134 [==============================] - 105s 781ms/step - loss: 4.2651e-05
Epoch 10/20
134/134 [==============================] - 104s 775ms/step - loss: 3.4638e-05
Epoch 11/20
134/134 [==============================] - 104s 776ms/step - loss: 2.8650e-05
Epoch 12/20
134/134 [==============

In [18]:
model.save("DNN1.h5")
